## O modelo de validação será feito usando a base de dados
## do projeto machine learning com classificação, a validação começa apartir do topico 1

# Bibliotecas

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier

## Carregamento dos dados

In [27]:
#leitura dos dados, apague também a coluna Unnnamed que estava em branco

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


## Modelo de Classificação

In [28]:
#separo os dados como treino e teste para meu modelo X E Y 

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))


Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [29]:
#DummyClassifier: É um modelo classificador que faz previsões sem tentar encontrar padrões nos dados.
#O modelo padrão basicamente analisa qual rótulo é mais frequente no conjunto de dados de treinamento e faz previsões com base nesse rótulo
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


In [30]:
#rodo uma arvore de decisao com profundidade 2 para entender os dados 
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


A acurácia foi 71.92%


# Cross validate

- É uma técnica muito utilizada para avaliação de desempenho de modelos e consequente verificação de tendências no aprendizado de máquina
- também conhecido como validação cruzada o cross valite é um metodo reamostragem que usa diferentes porções dos dados para testar e treinar um modelo em diferentes iterações
-  Essa validação é feita quebrando os dados em K-fold testando e treinando diversas vezes e comparando esses resultados um com os outros
- e com isso possibilitando a geração de um intervalo de confiança entre os resultados

In [31]:
from sklearn.model_selection import cross_validate

#aqui eu aplico o mesmo modelo de arvore de decisao do projeto acima
# sem alterar os  dados de treino e teste pois a ideia é que o cross validate quebre esses dados 
#treino e teste de acordo com as vezes que eu pedir, nesse ponto também eu chamo a media
# e o desvio padrao dos dados para que eu possar gerar o intervalo de confiança entre os resultados


modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score = False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy [74.99, 76.57]


In [32]:
#para garantir a reprodutividade do modelo adiciono o seed que estava acima
#o interesse é que apesar do seed mudar aqui os resultados do teste 
#continua o mesmo isso por conta do cross validate como eu estou aplicando
#3 vezes o teste o resultado já é diferente

SEED = 14
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score = False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy [74.99, 76.57]


### Apesar do Seed não estar mais alterando os resultados como anteriormente posso ficar 
### a mercer dos resultados quanto eu observo o numero de vezes que eu peço para treinar o modelo com cross validate
### como no exemplo abaixo, eu mudei de cv=3 para cv= 6 isso altero os dados e gero aleatoriedade
### nesse caso eu vou seguir a orientação da documentação e irei manter 5
### link documentação cross validate ( https://scikit-learn.org/stable/modules/cross_validation.html)

In [33]:
#exemplo 
SEED = 14
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 6, return_train_score = False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy [74.22, 77.34]


In [34]:
#vou manter por padrao 5

# e com isso encontro meu intervalo de desvio padrao

SEED = 14
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score = False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy [75.21, 76.35]


# Kfold com aleatorização

#### Nesse momento eu quero começa a rodar o modelo de uma maneira aleatoria, antes do 
#### algoritimo rodar o cross validate, pra isso vou usar o class sklearn.model_selection.KFold 
#### que faz essa aleatoriedade antes da validacao 

In [40]:
#abro uma função para fica mais chamar em chamar essas info da media, desvio padrao...

def imprime_resultado(results):
    media = results ['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [42]:
#chamo a biblioteca do KFOLD e junto dela dois parametros 
#n_splits o numero de vezes que eu quero que ele quebre  e embaralhe
# e suffle true que é aleatoriedade dos valores

from sklearn.model_selection import KFold


SEED = 14
np.random.seed(SEED)

cv= KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score = False)
imprime_resultado(results)

Accuracy [75.21, 76.35]


#  Estratificação com validação cruzada

#### Pode ocorrer no modelo de ML o desbalanceamento das classes de entradas ou saidas ou até mesmo pela natureza dos meus dados 
#### para fazer esse rebalançamento eu posso usar o  StratifiedKFold, que recebe um número de n_splits, shuffle e depois separa, mantendo a proporção entre as classes. 

Simulação de dados desbalanceados 

In [49]:
#pego a base de dadps original, faço a ordenação de proposito
#trago as mesmas classes 

dados_desbalanceados = dados.sort_values("vendido", ascending=True)
x_desbalanceados = dados_desbalanceados[["preco", "idade_do_modelo", "km_por_ano"]]
y_desbalanceados = dados_desbalanceados["vendido"]
dados_desbalanceados.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [56]:
#vou rodar o modelo agora sem o shufee 
#shuffe como coloquei acima é o que da aleatoriedade aos meus dados 


SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_desbalanceados,y_desbalanceados, cv = cv, return_train_score=False)
imprime_resultado(results)

Accuracy [34.29, 81.39]


In [53]:
#rodei o modelo agora com o shuffle
# então aqui eu consigo ver que o shufffe é bom quando 
# porém aqui eu não tenho uma separacao igualidaria das classes

SEED = 14
np.random.seed(SEED)

cv= KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_desbalanceados,y_desbalanceados, cv = 5, return_train_score = False)
imprime_resultado(results)

Accuracy [74.38, 77.18]


In [59]:
#aqui eu chamo o Stratified para fazer esse balanceamento 
# nele consigo passar 3 parametros
# stratifiedfold para o balanceiamento, o n_splites 
#para teste e o shuffle para aleatoriedade

from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_desbalanceados, y_desbalanceados, cv = cv, return_train_score=False)
imprime_resultado(results)

Accuracy [73.55, 78.01]
